# Load data

In [1]:
import numpy as np 
from monty.json import MontyDecoder
from monty.serialization import loadfn

data = loadfn('./bulk_moduli.json', cls=MontyDecoder)
structures = data['structures']
targets = np.log10(data['bulk_moduli'])

# Set up model and train

In [2]:
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GaussianDistance
from megnet.model import megnet_model

model = megnet_model(10, 2, n_blocks=1, lr=1e-2,
                     n1=4, n2=4, n3=4, n_pass=1, n_target=1,
                     graph_convertor=CrystalGraph(),
                     distance_convertor=GaussianDistance(np.linspace(0, 5, 10), 0.5))

Using TensorFlow backend.


In [3]:
model.train(structures, targets, epochs=2)

Epoch 1/2
1/1 [==============================] - 2s 2s/step - loss: 0.9996
Epoch 2/2
1/1 [==============================] - 0s 50ms/step - loss: 0.9979


# Prediction

## Predict from structure

In [4]:
from pymatgen import MPRester
mpr = MPRester()

# let's get some structure
structure = mpr.get_structure_by_material_id('mp-2340')


In [5]:
predicted_K = 10**model.predict_structure(structure).ravel()

In [6]:
print('The predicted K for {} is {} GPa'.format(structure.formula, predicted_K[0]))

The predicted K for Na6 O6 is 91.50774383544922 GPa
